In [1]:
# Getting input from a file
start_states = []
goal_states = []
heuristics = {}

with open("input_file.txt") as f:
        for (i,line) in enumerate(f):
            if(i==0):
                list1 = line.split(':')[1:]
                start_states = [list(map(int, state.split(','))) for state in list1]
            if(i==1):
                list1 = line.split(':')[1:]
                goal_states = [list(map(int, state.split(','))) for state in list1]
            if(i==2):
                list1 = (line.split(':'))[1].split(',')
                heuristics = {list1[0]:1,list1[1]:2}


In [2]:
print("Start states:",start_states,"\nGoal:",goal_states)
print("heuristic functions:",heuristics)

Start states: [[4, 5, 1, 2, 3, 8, 0, 6, 7], [0, 1, 3, 5, 2, 6, 4, 7, 8], [1, 2, 3, 5, 6, 0, 7, 8, 4]] 
Goal: [[1, 2, 3, 4, 5, 6, 7, 8, 0], [1, 2, 3, 5, 8, 6, 0, 7, 4]]
heuristic functions: {'Misplaced Tiles': 1, 'Manhattan Distance': 2}


In [3]:
class Puzzle:
    
    goal_state=None
    hn = None
    hn_value=None
    fn_value=None
    
    def __init__(self,state,parent,action,path_cost):     # initiater of class object
        
        self.state = state
        self.parent = parent
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        
        if self.hn == 1 or self.hn == 11:
            self.manhattan_hn()
        elif self.hn == 2 or self.hn == 22:
            self.misplaced_hn()
        
        if self.hn == 1 or self.hn == 2:
            self.fn_value = self.hn_value+self.path_cost
        else:
            self.fn_value = self.hn_value

        
    def __str__(self):                        #representing class object as customized string
        
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])

    
    def manhattan_hn(self):                   #manhattan distance heuristic calculation
        
        self.hn_value=0
        for num in range(1,9):
            distance = abs(self.state.index(num) - self.goal_state.index(num))
            i = int(distance/3)
            j = int(distance%3)
            self.hn_value = self.hn_value +i +j
        
        
    def misplaced_hn(self):                    #no. of misplaced tiles heuristic calculation
        
        sum = 0
        for i in range(9):
            if(self.state[i] == 0): continue
            if(self.state[i] != self.goal_state[i]): sum+=1
        self.hn_value = sum

        
    def goal_test(self):                       #check goal_state reached or not
        
        if self.state == self.goal_state:
            return True
        return False

    
    @staticmethod
    def find_legal_actions(i,j):               #finds legal actions from a particular tile in a puzzle
        
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:                             # up is illegal
            legal_action.remove('U')
        elif i == 2:                           # down is illegal
            legal_action.remove('D')
        if j == 0:                             # left is illegal
            legal_action.remove('L')
        elif j == 2:                           # right is illegal
            legal_action.remove('R')
        return legal_action

    
    def generate_child(self):                   #generate childs of state
        
        children=[]
        x = self.state.index(0)                 #getting index of element 0 i.e blank tile in linear array
        i = int(x / 3)
        j = int(x % 3)
        legal_actions = self.find_legal_actions(i,j)

        for action in legal_actions:            #for every legal action creating a child with new state
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x-3] = new_state[x-3], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x+3] = new_state[x+3], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]
            children.append(Puzzle(new_state,self,action,1))
        return children

    
    def find_solution(self):                    #find optimal path if goal_state reached
        
        solution = []
        solution.append(self.action)            #storing the action taken to come to this current state
        path = self
        while path.parent != None:              #backtracking to the start_state
            path = path.parent         
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution

In [4]:
from queue import PriorityQueue

def Astar(initial_state,goal_state,hn):
    
    Puzzle.goal_state = goal_state               #setting the value of class variable goal_state
    Puzzle.hn = hn                               #setting the value of class variable hn(heuristic function preference)
    count=0                                      #for counting no of states generated
    explored=[]                                  #list of states explored i.e. present in closed list
    start_node=Puzzle(initial_state,None,None,0) #initializing start_state object
    goal_node=Puzzle(goal_state,None,None,0)
    q = PriorityQueue()                          #creating open list as a priority queue
    q.put((start_node.fn_value,count,start_node))#inserting triplet(fn_value,count,state)

    t0 = time()
    
    while not q.empty():
        node=q.get()                             #removing element from queue         
        #print(node[0])
        node=node[2]                             #getting the third triplet 'state'
        explored.append(node.state)              #inserting the state into closed/explored state
        if node.goal_test():                     #checking the state for goal_state
            print("Puzzle solved successfully by A*\n")
            print("Start State\n",start_node,"\n")
            print("Goal State\n",node,"\n")
            print('Toatal states generated :',count)
            print('Total no. of states explored by A* : ',len(explored))
            opt_path = node.find_solution()
            print('Total number of states to optimal path: ',len(opt_path))
            print('Optimal path:',opt_path)
            print('Optimal path cost: ',node.path_cost)
            return
        elif time()-t0>20:
            print("Start State\n",start_node,"\n")
            print("Goal State\n",goal_node,"\n")
            print("Puzzle not solvable within time limit")
            print('Total no. of states explored by A* before termination : ',len(explored))
            return
        
        children=node.generate_child()          #generating childs of state if its not a goal state
        for child in children:                  # for every child if not explored, insert it in the open list
            if child.state not in explored:
                count += 1
                q.put((child.fn_value,count,child))
    return

In [5]:
def BFS(initial_state,goal_state,hn):
    
    Puzzle.goal_state = goal_state               #setting the value of class variable goal_state
    Puzzle.hn = hn                               #setting the value of class variable hn(heuristic function preference)
    count=0                                      #for counting no of states generated
    explored=[]                                  #list of states explored i.e. present in closed list
    start_node=Puzzle(initial_state,None,None,0) #initializing start_state object
    goal_node=Puzzle(goal_state,None,None,0)
    q = PriorityQueue()                          #creating open list as a priority queue
    q.put((start_node.fn_value,count,start_node))#inserting triplet(fn_value,count,state)

    t0 = time()
    
    while not q.empty():
        node=q.get()                             #removing element from queue         
        #print(node[0])
        node=node[2]                             #getting the third triplet 'state'
        explored.append(node.state)              #inserting the state into closed/explored state
        if node.goal_test():                     #checking the state for goal_state
            print("Puzzle solved successfully by BFS\n")
            print("Start State\n",start_node,"\n")
            print("Goal State\n",node,"\n")
            print('Toatal states generated :',count)
            print('Total no. of states explored by BFS : ',len(explored))
            opt_path = node.find_solution()
            print('Total number of states to optimal path: ',len(opt_path))
            print('Optimal path:',opt_path)
            print('Optimal path cost: ',node.path_cost)
            return
        elif time()-t0>20:
            print("Start State\n",start_node,"\n")
            print("Goal State\n",goal_node,"\n")
            print("Puzzle not solvable within time limit")
            print('Total no. of states explored by BFS before termination : ',len(explored))
            return
        
        children=node.generate_child()          #generating childs of state if its not a goal state
        for child in children:                  # for every child if not explored, insert it in the open list
            if child.state not in explored:
                count += 1
                q.put((child.fn_value,count,child))
    return

In [6]:
from time import time

start_state = start_states[0]
goal_state = goal_states[0]

print("*********************Manhattan Heuristic function used***********************\n")
t0 = time()
Astar(start_state,goal_state,1)
t11 = time() - t0
BFS(start_state,goal_state,11)
t12 = time() - t0 -t11
print('Time taken for Astar: ', t11)
print('Time taken for BFS: ', t12)

print()
print("*********************Misplaced Heuristic function used***********************\n")
Astar(start_state,goal_state,2)
t21 = time() - t0 -t11 -t12
BFS(start_state,goal_state,22)
t22 = time() - t0 -t11 -t12 -t21
print('Time taken for Astar: ', t21)
print('Time taken for BFS: ', t22)

*********************Manhattan Heuristic function used***********************

Puzzle solved successfully by A*

Start State
 [4, 5, 1]
[2, 3, 8]
[0, 6, 7] 

Goal State
 [1, 2, 3]
[4, 5, 6]
[7, 8, 0] 

Toatal states generated : 2085
Total no. of states explored by A* :  1291
Total number of states to optimal path:  22
Optimal path: ['U', 'R', 'D', 'R', 'U', 'L', 'L', 'D', 'R', 'R', 'U', 'L', 'U', 'R', 'D', 'L', 'L', 'U', 'R', 'D', 'R', 'D']
Optimal path cost:  22
Puzzle solved successfully by BFS

Start State
 [4, 5, 1]
[2, 3, 8]
[0, 6, 7] 

Goal State
 [1, 2, 3]
[4, 5, 6]
[7, 8, 0] 

Toatal states generated : 170
Total no. of states explored by BFS :  98
Total number of states to optimal path:  40
Optimal path: ['R', 'R', 'U', 'L', 'U', 'R', 'D', 'L', 'L', 'U', 'R', 'D', 'L', 'D', 'R', 'U', 'L', 'D', 'R', 'R', 'U', 'L', 'L', 'D', 'R', 'R', 'U', 'L', 'D', 'L', 'U', 'R', 'R', 'D', 'L', 'U', 'L', 'D', 'R', 'R']
Optimal path cost:  40
Time taken for Astar:  0.14505958557128906
Time taken 